In [2]:
#https://medium.com/@sabber/classifying-yelp-review-comments-using-lstm-and-word-embeddings-part-1-eb2275e4066b
path = "/Users/KumarSanjeev/Downloads/sentiment labelled sentences/yelp_labelled.txt"
path

'/Users/KumarSanjeev/Downloads/sentiment labelled sentences/yelp_labelled.txt'

In [3]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [4]:
data=pd.read_csv(path,sep="\t",names=["Tweet","Sentiment"])
data.head()

,Tweet,Sentiment
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [5]:
X=data["Tweet"]
y=data["Sentiment"]

In [6]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.33)

In [17]:
len(X_train),len(y_train)

(670, 670)

In [18]:
len(X_test),len(y_test)

(330, 330)

In [20]:
vectorizer=TfidfVectorizer()
X_train=vectorizer.fit_transform(X_train)

In [26]:
X_train.shape

(670, 1616)

In [29]:
X_test=vectorizer.transform(X_test)

In [30]:
X_test.shape

(330, 1616)

In [31]:
from sklearn.linear_model import LogisticRegression
model=LogisticRegression()
model.fit(X_train,y_train)

/Users/kumarsanjeev/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [34]:
y_pred=model.predict(X_test)

In [35]:
score= accuracy_score(y_pred,y_test)

In [36]:
score

0.796969696969697

In [7]:
from keras.models import Sequential
from keras.layers import Dense

/Users/kumarsanjeev/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [8]:
print("importing libraries.....")


import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding,SpatialDropout1D
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
%matplotlib inline

importing libraries.....


In [9]:
model1=Sequential()

In [10]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.15)

In [21]:
max_words = 25000
max_len = 150
tok = Tokenizer(num_words=max_words)


In [23]:
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)
sequences_matrix

array([[   0,    0,    0, ...,  171,   26,   75],
       [   0,    0,    0, ...,  810,  318,  229],
       [   0,    0,    0, ...,   37,   40,  811],
       ...,
       [   0,    0,    0, ...,   28,   33, 1867],
       [   0,    0,    0, ...,  115,   72,  385],
       [   0,    0,    0, ...,  806,  292,  510]], dtype=int32)

In [13]:
model = Sequential()
model.add(Embedding(max_words,300, input_length=max_len))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [14]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 150, 300)          7500000   
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               160400    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 7,660,501
Trainable params: 7,660,501
Non-trainable params: 0
_________________________________________________________________


In [15]:
model.fit(sequences_matrix,y_train,batch_size=128,epochs=10,
          validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

Train on 680 samples, validate on 170 samples
Epoch 1/10
680/680 [==============================] - 4s 6ms/step - loss: 0.6916 - acc: 0.5574 - val_loss: 0.6898 - val_acc: 0.5647
Epoch 2/10
680/680 [==============================] - 2s 4ms/step - loss: 0.6740 - acc: 0.7103 - val_loss: 0.6887 - val_acc: 0.5059
Epoch 3/10
680/680 [==============================] - 3s 4ms/step - loss: 0.6366 - acc: 0.7147 - val_loss: 0.6699 - val_acc: 0.5824
Epoch 4/10
680/680 [==============================] - 2s 4ms/step - loss: 0.5505 - acc: 0.8515 - val_loss: 0.6154 - val_acc: 0.7000
Epoch 5/10
680/680 [==============================] - 3s 4ms/step - loss: 0.4375 - acc: 0.8721 - val_loss: 0.6219 - val_acc: 0.6353


In [16]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

In [17]:
accr = model.evaluate(test_sequences_matrix,y_test)
accr

150/150 [==============================] - 0s 1ms/step


[0.5523098889986674, 0.686666669845581]

In [18]:
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

Test set
  Loss: 0.552
  Accuracy: 0.687


In [48]:
input_dim = X_train.shape[1]  # Number of features
model1.add(Dense(100,input_dim=input_dim,activation="relu"))
model1.add(Dense(1,activation="sigmoid"))

In [49]:
model1.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])

In [50]:
model1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 100)               161700    
_________________________________________________________________
dense_2 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 101       
_________________________________________________________________
dense_4 (Dense)              (None, 100)               200       
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 101       
Total params: 172,202
Trainable params: 172,202
Non-trainable params: 0
_________________________________________________________________


In [53]:
history=model1.fit(X_train,y_train,epochs=25,validation_data=[X_test,y_test],batch_size=25)

Train on 670 samples, validate on 330 samples
Epoch 1/25
670/670 [==============================] - 0s 239us/step - loss: 0.0708 - acc: 0.9985 - val_loss: 0.4593 - val_acc: 0.8000
Epoch 2/25
670/670 [==============================] - 0s 175us/step - loss: 0.0522 - acc: 0.9985 - val_loss: 0.4641 - val_acc: 0.8152
Epoch 3/25
670/670 [==============================] - 0s 175us/step - loss: 0.0396 - acc: 0.9985 - val_loss: 0.4839 - val_acc: 0.8182
Epoch 4/25
670/670 [==============================] - 0s 170us/step - loss: 0.0299 - acc: 1.0000 - val_loss: 0.5290 - val_acc: 0.8061
Epoch 5/25
670/670 [==============================] - 0s 168us/step - loss: 0.0245 - acc: 1.0000 - val_loss: 0.5498 - val_acc: 0.8152
Epoch 6/25
670/670 [==============================] - 0s 197us/step - loss: 0.0199 - acc: 1.0000 - val_loss: 0.5730 - val_acc: 0.8152
Epoch 7/25
670/670 [==============================] - 0s 170us/step - loss: 0.0169 - acc: 1.0000 - val_loss: 0.5776 - val_acc: 0.8152
Epoch 8/25
670/6

In [58]:
loss,accuracy=model1.evaluate(X_train,y_train)
print("Training Accuracy: {:.5f}".format(accuracy))

loss,accuracy=model1.evaluate(X_test,y_test)
print("Training Accuracy: {:.4f}".format(accuracy))

670/670 [==============================] - 0s 45us/step
Training Accuracy: 1.00000
0.002734439439181961 1.0
330/330 [==============================] - 0s 53us/step
Training Accuracy: 0.8121


In [64]:
import spacy
import en_core_web_sm
nlp=en_core_web_sm.load()

In [74]:
from pprint import pprint
doc=nlp("Watch live stream of the ICC U-19 World Cup 2020 semi-final between India U-19 and Pakistan U-19")
a= [(token,token.pos_) for token in doc]
pprint(a)

[(Watch, 'VERB'),
 (live, 'ADJ'),
 (stream, 'NOUN'),
 (of, 'ADP'),
 (the, 'DET'),
 (ICC, 'PROPN'),
 (U-19, 'PROPN'),
 (World, 'PROPN'),
 (Cup, 'PROPN'),
 (2020, 'NUM'),
 (semi, 'NOUN'),
 (-, 'PUNCT'),
 (final, 'ADJ'),
 (between, 'ADP'),
 (India, 'PROPN'),
 (U-19, 'PROPN'),
 (and, 'CCONJ'),
 (Pakistan, 'PROPN'),
 (U-19, 'PROPN')]
